In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("taxi-duration-prediction")\
            .config("spark.executor.memory", MAX_MEMORY)\
            .config("spark.driver.memory", MAX_MEMORY).getOrCreate()

In [3]:
trip_files = "/Users/keon/fastcampus/data-engineering/01-spark/data/trips/*"

In [4]:
trips_df = spark.read.csv(f"file:///{trip_files}", inferSchema=True, header=True)

In [5]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
trips_df.createOrReplaceTempView("trips")

In [7]:
query = """
SELECT
    trip_distance,
    duration
FROM
    (SELECT
        *,
        TO_DATE(t.tpep_pickup_datetime) AS pickup_date,
        UNIX_TIMESTAMP(t.tpep_dropoff_datetime) - UNIX_TIMESTAMP(t.tpep_pickup_datetime) AS duration
    FROM
        trips t)
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND pickup_date >= '2021-01-01'
    AND pickup_date < '2021-08-01'
    AND duration > 0
"""
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [8]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=1)
print(train_df.count())
print(test_df.count())

10499761
2625821


In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
vassembler = VectorAssembler(inputCols=['trip_distance'], outputCol='features')
vtrain_df = vassembler.transform(train_df)
vtrain_df.show()

+-------------+--------+--------+
|trip_distance|duration|features|
+-------------+--------+--------+
|         0.01|       2|  [0.01]|
|         0.01|       2|  [0.01]|
|         0.01|       2|  [0.01]|
|         0.01|       2|  [0.01]|
|         0.01|       2|  [0.01]|
|         0.01|       2|  [0.01]|
|         0.01|       2|  [0.01]|
|         0.01|       2|  [0.01]|
|         0.01|       3|  [0.01]|
|         0.01|       3|  [0.01]|
|         0.01|       3|  [0.01]|
|         0.01|       3|  [0.01]|
|         0.01|       3|  [0.01]|
|         0.01|       3|  [0.01]|
|         0.01|       3|  [0.01]|
|         0.01|       3|  [0.01]|
|         0.01|       4|  [0.01]|
|         0.01|       4|  [0.01]|
|         0.01|       4|  [0.01]|
|         0.01|       4|  [0.01]|
+-------------+--------+--------+
only showing top 20 rows



In [11]:
from pyspark.ml.regression import LinearRegression

In [12]:
lr = LinearRegression(maxIter=50,
                      labelCol="duration",
                      featuresCol="features")

In [13]:
model = lr.fit(vtrain_df)

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator

In [15]:
vtest_df = vassembler.transform(test_df)

In [16]:
predictions = model.transform(vtest_df).cache()

In [17]:
predictions.show()

+-------------+--------+--------+----------------+
|trip_distance|duration|features|      prediction|
+-------------+--------+--------+----------------+
|         0.01|       2|  [0.01]|538.494134101831|
|         0.01|       3|  [0.01]|538.494134101831|
|         0.01|       3|  [0.01]|538.494134101831|
|         0.01|       3|  [0.01]|538.494134101831|
|         0.01|       3|  [0.01]|538.494134101831|
|         0.01|       4|  [0.01]|538.494134101831|
|         0.01|       4|  [0.01]|538.494134101831|
|         0.01|       4|  [0.01]|538.494134101831|
|         0.01|       4|  [0.01]|538.494134101831|
|         0.01|       5|  [0.01]|538.494134101831|
|         0.01|       5|  [0.01]|538.494134101831|
|         0.01|       5|  [0.01]|538.494134101831|
|         0.01|       5|  [0.01]|538.494134101831|
|         0.01|       6|  [0.01]|538.494134101831|
|         0.01|       6|  [0.01]|538.494134101831|
|         0.01|       7|  [0.01]|538.494134101831|
|         0.01|       7|  [0.01

In [18]:
from pyspark.sql.types import DoubleType
distance_list = [1.1, 5.5, 10.5, 30.0]
distances_df = spark.createDataFrame(distance_list, DoubleType()).toDF("trip_distance")

In [19]:
distances_df.show()

+-------------+
|trip_distance|
+-------------+
|          1.1|
|          5.5|
|         10.5|
|         30.0|
+-------------+



In [20]:
vdistances_df = vassembler.transform(distances_df)

In [21]:
vdistances_df.show()

+-------------+--------+
|trip_distance|features|
+-------------+--------+
|          1.1|   [1.1]|
|          5.5|   [5.5]|
|         10.5|  [10.5]|
|         30.0|  [30.0]|
+-------------+--------+



In [22]:
model.transform(vdistances_df).show()

+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          1.1|   [1.1]| 676.9693698522087|
|          5.5|   [5.5]| 1235.951972881256|
|         10.5|  [10.5]|1871.1594763233556|
|         30.0|  [30.0]| 4348.468739747544|
+-------------+--------+------------------+



# 성능 평가

In [23]:
model.summary

In [24]:
print("RMSE: ", model.summary.rootMeanSquaredError)

RMSE:  3528.3761929145353


In [25]:
print("RMSE: ", model.summary.r2)

RMSE:  0.018565176935511962


In [26]:
train_df.describe().show()

+-------+------------------+-----------------+
|summary|     trip_distance|         duration|
+-------+------------------+-----------------+
|  count|          10499761|         10499761|
|   mean|2.8817192743725615|903.3216602739815|
| stddev|3.8198657789721615|3561.592040114176|
|    min|              0.01|                1|
|    max|             452.0|          1729062|
+-------+------------------+-----------------+

